In [1]:
!pip install nltk
!pip install tensorflow
!pip install numpy

In [27]:
import numpy as np
import json
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
lemmatizer = WordNetLemmatizer()

# Load the intents file
with open('intents.json') as file:
    intents = json.load(file)

words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Tokenize each pattern
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

print(f"Documents: {documents}")
print(f"Classes: {classes}")
print(f"Words: {words}")


Documents: [(['Hi'], 'greeting'), (['Hello'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Good', 'day'], 'greeting'), (['Hey'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Take', 'care'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thanks', 'a', 'lot'], 'thanks'), (['How', 'are', 'you'], 'state'), (['hru'], 'state'), (['sup'], 'state'), (['How', 'are', 'you', 'doing', 'lately', '?'], 'state'), (['Are', 'you', 'okay', '?'], 'state'), (['I', 'like', 'you', '!'], 'like'), (['looool'], 'like'), (['You', 'are', 'fun', '!'], 'like'), (['What', 'is', 'your', 'name', '?'], 'name'), (['Can', 'you', 'tell', 'me', 'your', 'name', '?'], 'name'), (['What', 'are', 'you', '?'], 'who'), (['Who', 'are', 'you', '?'], 'who'), (['Can', 'you', 'tell', 'me', 'who', 'you', 'are', '?'], 'who'), (['How', 'old', 'are', 'you', '?'], 'age'

In [29]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))


In [30]:
from tensorflow.keras.optimizers.legacy import SGD

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')


Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7466 - accuracy: 0.1400
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7199 - accuracy: 0.0800
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.6597 - accuracy: 0.1200
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.5791 - accuracy: 0.1600
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.5663 - accuracy: 0.1400
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 2.5324 - accuracy: 0.2400
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 2.5059 - accuracy: 0.2000
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 2.4424 - accuracy: 0.1800
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 2.4383 - accuracy: 0.1800
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 2.3318 - accuracy: 0.2400
Epoch 11/

In [31]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("Chatbot is ready to respond!")


Chatbot is ready to respond!


In [ ]:
t

In [32]:
while True:
    message = input("You: ")
    if message.lower() in ['bye', 'see you later', 'goodbye', 'take care']:
        print("Bot: " + random.choice(intents['intents'][1]['responses']))  # Response from "goodbye" intent
        break

    ints = predict_class(message, model)
    res = get_response(ints, intents)
    print(f"Bot: {res}")


You: hi


1/1 [==============================] - 0s 61ms/step
Bot: Good day! How can I assist you?
You: who are you
1/1 [==============================] - 0s 21ms/step
Bot: My name is Chatty!
You: what time is it ?
1/1 [==============================] - 0s 21ms/step
Bot: Try to check the clock !
You: loool
1/1 [==============================] - 0s 21ms/step
Bot: Hi! How can I help you?
You: loool
1/1 [==============================] - 0s 21ms/step
Bot: Good day! How can I assist you?
You: looool
1/1 [==============================] - 0s 20ms/step
Bot: :)
You: 
1/1 [==============================] - 0s 20ms/step
Bot: Hello!
You: i like ytou
1/1 [==============================] - 0s 21ms/step
Bot: UwU
You: what do you offer
1/1 [==============================] - 0s 20ms/step
Bot: I offer support with a range of inquiries and tasks.
You: what do you offer
1/1 [==============================] - 0s 21ms/step
Bot: I can be the best person you can talk with
You: nice
1/1 [==============================

KeyboardInterrupt: Interrupted by user